In [146]:
## import data
import scipy
test = scipy.io.loadmat('ng207528x8165itst.mat')
test_data = test['Dtst'].tolist()[0][0][1]
test_label_matrix = test['Dtst'].tolist()[0][0][2]
train = scipy.io.loadmat('ng2011293x8165itrn.mat')
train_data = train['Dtrn'].tolist()[0][0][1]
train_label_matrix = train['Dtrn'].tolist()[0][0][2]

In [235]:
## transform label matrix to vectors
import numpy as np
sample = {}
subset = []
label = []
for i in range(20):
    sample_i = np.random.choice(np.where(train_label_matrix.todense()[:,i]!=0)[0],100, replace = False)
    sample[str(i)] = sample_i
    subset =np.hstack((subset,sample_i))
    label = np.hstack((label, np.repeat(i, 100)))
train_select = train_data[subset,]

In [236]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer()
# vectors = vectorizer.fit_transform(train_select)

In [237]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
solution = LDA(n_components=20, learning_method='batch', max_iter=60).fit_transform(train_select)

In [205]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=5)
pca.fit(solution)
X = pca.transform(solution)

In [238]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(solution, label)
clf.predict(solution)
sum(clf.predict(solution)==label)/2000.

0.45400000000000001

In [232]:
from scipy.sparse import csc_matrix
A = csc_matrix(train_select, dtype=float)
[u, s, vt] = scipy.sparse.linalg.svds(A, 20)
u.shape

(2000, 20)

In [233]:
X = np.dot(u, np.diag(s))
clf = svm.SVC(kernel='linear')
clf.fit(X, label)
sum(clf.predict(X)==label)/2000.

0.53649999999999998

In [259]:
# random sample SVM. Data matrix: X, n * p, n = 2000, 100 sample in each catogory
# n: sample size in each category; k: number of class
def random_sample(n, k, m):
    sample = []
    for i in range(k):
        sample_i = np.random.choice(range(i*n, (i+1)*n), size = m, replace=False)
        sample = np.hstack((sample, sample_i))
    sample = np.array(sample, dtype=int)
    return(sample)

In [266]:
S = 0
for i in range(20):
    labeled_data = random_sample(100, 20, 5)
    X = np.dot(u, np.diag(s))
    clf = svm.SVC(`kernel='linear')
    clf.fit(X[labeled_data,], label[labeled_data])
    S += sum(clf.predict(X)==label)/2000.
S/20.

0.28482499999999999

In [276]:
## SVD + SVM
label_num = 20
from scipy.sparse import csc_matrix
from sklearn import svm
A = csc_matrix(train_select, dtype=float)
TOPIC_NUM = [5,10,20,25,30,40,50]
for topic_num in TOPIC_NUM:
    [u, s, vt] = scipy.sparse.linalg.svds(A, topic_num)
    X = np.dot(u, np.diag(s))
    S = 0
    for i in range(20):
        labeled_data = random_sample(100, 20, label_num)
        clf = svm.SVC(kernel='linear')
        clf.fit(X[labeled_data,], label[labeled_data])
        S += sum(clf.predict(X)==label)/2000.
    print([label_num, topic_num, round(S/20.,4)])

[20, 5, 0.2067]
[20, 10, 0.3478]
[20, 20, 0.446]
[20, 25, 0.4697]
[20, 30, 0.4814]
[20, 40, 0.5102]
[20, 50, 0.5273]


In [282]:
## LDA + SVM
label_num = 15
from scipy.sparse import csc_matrix
from sklearn import svm
from sklearn.decomposition import LatentDirichletAllocation as LDA
TOPIC_NUM = [5,10,20,25,30,40,50]
for topic_num in TOPIC_NUM:
    solution = LDA(n_components=topic_num, learning_method='batch', max_iter=60).fit_transform(train_select)
    S = 0
    for i in range(20):
        labeled_data = random_sample(100, 20, label_num)
        clf = svm.SVC(kernel='linear')
        clf.fit(X[labeled_data,], label[labeled_data])
        accuracy = sum(clf.predict(X)==label)/2000.
        S += accuracy
    print([label_num, topic_num, round(S/20.,4)])

[15, 5, 0.4838]
[15, 10, 0.4831]
[15, 20, 0.4853]
[15, 25, 0.4887]
[15, 30, 0.4855]
[15, 40, 0.4794]
[15, 50, 0.4863]


** SVD + SVM, TRAIN_ACCURACY:

[LABEL_NUM, TOPIC_NUM, ACCURACY]
[5, 5, 0.1555]
[5, 10, 0.2539]
[5, 20, 0.31]
[5, 25, 0.3207]
[5, 30, 0.3224]
[5, 40, 0.3403]
[5, 50, 0.3358]

[10, 5, 0.1833]
[10, 10, 0.3021]
[10, 20, 0.3786]
[10, 25, 0.4066]
[10, 30, 0.4006]
[10, 40, 0.4239]
[10, 50, 0.4244]

[15, 5, 0.2008]
[15, 10, 0.3288]
[15, 20, 0.4199]
[15, 25, 0.4421]
[15, 30, 0.4565]
[15, 40, 0.4725]
[15, 50, 0.4878]

[20, 5, 0.2067]
[20, 10, 0.3478]
[20, 20, 0.446]
[20, 25, 0.4697]
[20, 30, 0.4814]
[20, 40, 0.5102]
[20, 50, 0.5273]

** LDA + SVM, TRAIN_ACCURACY:

[LABEL_NUM, TOPIC_NUM, ACCURACY]
[5, 5, 0.3335]
[5, 10, 0.3396]
[5, 20, 0.338]
[5, 25, 0.3389]
[5, 30, 0.3426]
[5, 40, 0.3407]
[5, 50, 0.3354]

[10, 5, 0.4262]
[10, 10, 0.418]
[10, 20, 0.427]
[10, 25, 0.4296]
[10, 30, 0.4291]
[10, 40, 0.4324]
[10, 50, 0.4232]

[15, 5, 0.4838]
[15, 10, 0.4831]
[15, 20, 0.4853]
[15, 25, 0.4887]
[15, 30, 0.4855]
[15, 40, 0.4794]
[15, 50, 0.4863]

[20, 5, 0.5263]
[20, 10, 0.5293]
[20, 20, 0.5307]
[20, 25, 0.5292]
[20, 30, 0.5274]
[20, 40, 0.5282]
[20, 50, 0.5201]